<a href="https://colab.research.google.com/github/systemlogic/Algorithm/blob/master/Nvidia_TansorRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install torch_tensorrt
!wget -O /tmp/sparrow.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/House_Sparrow%2C_England_-_May_09.jpg/800px-House_Sparrow%2C_England_-_May_09.jpg

--2025-09-02 10:01:30--  https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/House_Sparrow%2C_England_-_May_09.jpg/800px-House_Sparrow%2C_England_-_May_09.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 185.15.59.240, 2a02:ec80:300:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|185.15.59.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112334 (110K) [image/jpeg]
Saving to: ‘/tmp/sparrow.jpg’

/tmp/sparrow.jpg    100%[===================>] 109.70K  --.-KB/s    in 0.01s   

2025-09-02 10:01:30 (8.59 MB/s) - ‘/tmp/sparrow.jpg’ saved [112334/112334]



In [16]:
import torch
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
import time
import torch.backends.cudnn as cudnn

img = Image.open("/tmp/sparrow.jpg")

topk = 3

model = models.resnet50(pretrained = True).to("cuda")
# img = Image.open(handler)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
img = transform(img)
img_batch = torch.unsqueeze(img, 0).to("cuda")
# Uncomment below two lines together to transfer the model as well as image both to mps.
# img_batch = img_batch.to(torch.device("mps"))
# model = model.to(torch.device("mps"))

model.eval()
with torch.no_grad():
    outputs = model(img_batch)
probs = torch.nn.functional.softmax(outputs[0], dim=0)

print("Input tensor shape:", img_batch.shape)

catagories = pd.read_csv("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", header = None)
prob, class_number = torch.topk(probs, topk)
for i in range(topk):
    probability = prob[i].item()
    class_name = catagories[0][int(class_number[i])]
    print("%{} {}".format(int(probability*100), class_name))



cudnn.benchmark = True

def benchmark(model, device="cuda", input_shape=(32, 3, 224, 224), dtype='fp32', nwarmup=50, nruns=100):
    input_data = torch.randn(input_shape)
    input_data = input_data.to(device)

    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            features = model(input_data)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            features = model(input_data)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, ave batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print("Output features size:", features.size())
    print('Average batch time: %.2f ms'%(np.mean(timings)*1000))

model = model
benchmark(model)

  warnings.warn(

  warnings.warn(msg)



Input tensor shape: torch.Size([1, 3, 224, 224])
%90 house finch
%4 junco
%2 brambling
Warm up ...
Start timing ...
Iteration 10/100, ave batch time 88.90 ms
Iteration 20/100, ave batch time 88.98 ms
Iteration 30/100, ave batch time 88.99 ms
Iteration 40/100, ave batch time 89.07 ms
Iteration 50/100, ave batch time 89.16 ms
Iteration 60/100, ave batch time 89.21 ms
Iteration 70/100, ave batch time 89.30 ms
Iteration 80/100, ave batch time 89.36 ms
Iteration 90/100, ave batch time 89.44 ms
Iteration 100/100, ave batch time 89.51 ms
Input shape: torch.Size([32, 3, 224, 224])
Output features size: torch.Size([32, 1000])
Average batch time: 89.51 ms


In [19]:
traced_model = torch.jit.trace(model, [torch.randn((1, 3, 224, 224)).to("cuda")])

import torch_tensorrt
import pandas as pd

catagories = pd.read_csv("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", header = None)



print("Input tensor shape:", img_batch.shape)

trt_model = torch_tensorrt.compile(
    traced_model,
    inputs = [torch_tensorrt.Input((1, 3, 224, 224), dtype=torch.float32)],
    enabled_precisions = {torch.float32}
)


trt_model.eval()
with torch.no_grad():
    outputs = trt_model(img_batch)
prob = torch.nn.functional.softmax(outputs[0], dim=0)

probs, classes = torch.topk(prob, topk)
for i in range(topk):
    probability = probs[i].item()
    class_label = catagories[0][int(classes[i])]
    print("%{} {}".format(int(probability*100), class_label))


Input tensor shape: torch.Size([1, 3, 224, 224])
%90 house finch
%4 junco
%2 brambling
